In [1]:
import dotenv
import logging
import sys
import uuid
from pprint import pprint

dotenv.load_dotenv()

logging.disable(sys.maxsize)

In [2]:
DIGICOACH_NAME = "Nieuwe digicoach TEST onder interactie"
TASK_NAME = "Nieuwe taak TEST onder interactie"

In [3]:
from askdelphi_client import AskDelphiClient

client = AskDelphiClient()

client.authenticate()

True

In [4]:
try:
    topic_type_map = {}

    design = client.get_content_design()

    topic_types = design.get("topicTypes", [])
    print(f"Found {len(topic_types)} topic types:\n")

    for i, tt in enumerate(topic_types[:100]):  # Show first 100
        if tt.get("title").lower() in ["action", "task", "digitale coach procespagina", "pre-defined search"]:
            print(f"  {i+1}. {tt.get('title', 'Unknown')}")
            print(f"     ID: {tt.get('key', 'N/A')}")
            print(f"     Namespace: {tt.get('namespace', 'N/A')}")
            print()

            topic_type_map[tt.get("title")] = tt.get("key")

    if len(topic_types) > 100:
        print(f"  ... and {len(topic_types) - 100} more")

except Exception as e:
    print(f"FAILED: {e}")

print(f"topic type map:\n")
print(topic_type_map)

Found 39 topic types:

  14. Pre-defined search
     ID: 72aaefbb-1c09-4f89-8830-54194b03523b
     Namespace: http://tempuri.org/imola-pre-defined-search

  21. Task
     ID: 6aba8437-c8df-42d2-a868-840847c124ca
     Namespace: http://tempuri.org/imola-task

  22. Digitale Coach Procespagina
     ID: 7d332fbb-44f5-469f-b570-874e701e526b
     Namespace: http://tempuri.org/imola-skill-area

  29. Action
     ID: c568af9a-6c89-45cf-a580-bc94e1c62ae3
     Namespace: http://tempuri.org/imola-action

topic type map:

{'Pre-defined search': '72aaefbb-1c09-4f89-8830-54194b03523b', 'Task': '6aba8437-c8df-42d2-a868-840847c124ca', 'Digitale Coach Procespagina': '7d332fbb-44f5-469f-b570-874e701e526b', 'Action': 'c568af9a-6c89-45cf-a580-bc94e1c62ae3'}


In [5]:
# 1. Gebruik topic version van voorgedefinieerde zoekopdracht om deze uit te checken, deze kun je vervolgens gebruiken om een digicoach aan te hangen.
topic_id_search = "9188fa3e-42c0-40dc-bf43-12fe2ee4e6d5"
endpoint = f"v1/tenant/{{tenantId}}/project/{{projectId}}/acl/{{aclEntryId}}/topic/{topic_id_search}/workflowstate"
result = client._request("POST", endpoint,
                json_data={
                    "action": 1
                })
topicVersionId_search = result["topicVersionId"]
print(topicVersionId_search)

393ac9d8-84aa-413f-9aad-0715652e10be


In [6]:
# Get relationship ID for digitale coach procespagina
task_relation_id = ""
endpoint = f"/v2/tenant/{{tenantId}}/project/{{projectId}}/acl/{{aclEntryId}}/topic/{topic_id_search}/topicVersion/{topicVersionId_search}/allowedrelations"
result = client._request("GET", endpoint,
                json_data={}
                )

for relation in result["topicAllowedRelations"]:
    if relation["topicTypeName"] == "Voorgedefinieerde zoekopdracht":
        search_relation_id = relation["relationTypeId"]
        break

search_relation_id

'de1ef13d-22e7-4ce2-9481-14167c451fd7'

In [7]:
# create digicoach met voorgedefineerde zoekopdracht als parent
topic_id_digicoach = str(uuid.uuid4())

result_1 = client._request(
    "POST",
    "v4/tenant/{tenantId}/project/{projectId}/acl/{aclEntryId}/topic",
    json_data={
        "topicId": topic_id_digicoach,
        "topicTitle": DIGICOACH_NAME,
        "topicTypeId": topic_type_map["Digitale Coach Procespagina"],
        "parentTopicId": topic_id_search,
        "parentTopicRelationTypeId": search_relation_id,
        "parentTopicVersionId": topicVersionId_search
        }
)

In [8]:
# Digicoach uitchecken, want nodig voor koppelen taak
endpoint = f"v1/tenant/{{tenantId}}/project/{{projectId}}/acl/{{aclEntryId}}/topic/{topic_id_digicoach}/workflowstate"
result = client._request("POST", endpoint,
                json_data={
                    "action": 1
                })
topicVersionId_digicoach = result["topicVersionId"]
print(topicVersionId_digicoach)

3f1a9c0b-290e-4112-ac81-3b117ff9ddbf


In [9]:
# get relationship ID for task
task_relation_id = ""
endpoint = f"/v2/tenant/{{tenantId}}/project/{{projectId}}/acl/{{aclEntryId}}/topic/{topic_id_digicoach}/topicVersion/{topicVersionId_digicoach}/allowedrelations"
result = client._request("GET", endpoint,
                json_data={}
                )

for relation in result["topicAllowedRelations"]:
    if relation["topicTypeName"] == "Taak":
        task_relation_id = relation["relationTypeId"]
        break

task_relation_id


'1de7ca2f-d566-4245-96f6-f4cc56434685'

In [10]:
# Aanmaken nieuwe taak met digicoach als parent
topic_id_task = str(uuid.uuid4())

result_2 = client._request(
    "POST",
    "v4/tenant/{tenantId}/project/{projectId}/acl/{aclEntryId}/topic",
    json_data={
        "topicId": topic_id_task,
        "topicTitle": TASK_NAME,
        "topicTypeId": topic_type_map["Task"],
        "parentTopicId": topic_id_digicoach,
        "parentTopicRelationTypeId": task_relation_id,
        "parentTopicVersionId": topicVersionId_digicoach
        }
)

In [ ]:
# Inchecken van search + digicoach, want eerder uitgecheckt om digicoach + taak te koppelen
# action 0 == inchecken
endpoint = f"v1/tenant/{{tenantId}}/project/{{projectId}}/acl/{{aclEntryId}}/topic/{topic_id_digicoach}/workflowstate"
result = client._request("POST", endpoint,
                json_data={
                    "action": 0
                })
topicVersionId_digicoach = result["topicVersionId"]

endpoint = f"v1/tenant/{{tenantId}}/project/{{projectId}}/acl/{{aclEntryId}}/topic/{topic_id_search}/workflowstate"
result = client._request("POST", endpoint,
                json_data={
                    "action": 0
                })
topicVersionId_search= result["topicVersionId"]


# HIERBOVEN BOUW IK OPNIEUW OP, ONDER is naslaagwerk

In [13]:
# Creates a workflow transition request for specifief topic
endpoint = f"v1/tenant/{{tenantId}}/project/{{projectId}}/acl/{{aclEntryId}}/workflow-transition-request/topic/{topic_id_search}"


# url is cms link naar het project
result = client._request("POST", endpoint,
                         json_data={
                             "Url": "https://digitalecoach.askdelphi.com/cms/tenant/0be6d42b-c278-44e6-888e-ba122840d690/project/397296f6-20dd-45cd-8459-250db2725140/acl/4ecd88f2-979b-4fb0-a95d-175d499bc375"
                         })

request_id_search = result["workflowTransitionRequestId"]

In [14]:
# Gets workflow transition request transitions model for specified request id
endpoint = f"v1/tenant/{{tenantId}}/project/{{projectId}}/acl/{{aclEntryId}}/workflow-transition-request/{request_id_search}/transitions"

result = client._request("GET", endpoint)
print(result)

{'data': {'selectedTransitions': [{'sequenceNo': 0, 'workflowId': 'd5dc5516-a8c7-43a9-bb59-c73dd1d0682b', 'workflowTitle': 'Default workflow', 'transitionId': 'f7adf76b-eda9-44c3-b5a5-08a9903e85a9', 'transitionTitle': 'Publiceer Concept naar Productie', 'sourceStageId': '45faa337-2499-4f06-a5c2-c919e4291bb2', 'sourceStageTitle': 'Concept', 'targetStageId': '9d3b3151-b543-4d6f-a9e5-18f4388753e2', 'targetStageTitle': 'Productie'}, {'sequenceNo': 1, 'workflowId': 'd5dc5516-a8c7-43a9-bb59-c73dd1d0682b', 'workflowTitle': 'Default workflow', 'transitionId': 'a4ece4c3-e9a3-4a9b-b163-338c2bb04d85', 'transitionTitle': 'Publiceer Concept naar Test', 'sourceStageId': '45faa337-2499-4f06-a5c2-c919e4291bb2', 'sourceStageTitle': 'Concept', 'targetStageId': 'ad70aeea-a938-469d-a6fb-493883ae982b', 'targetStageTitle': 'Test'}], 'existingTransitions': [{'workflowId': 'd5dc5516-a8c7-43a9-bb59-c73dd1d0682b', 'workflowTitle': 'Default workflow', 'transitionId': 'f7adf76b-eda9-44c3-b5a5-08a9903e85a9', 'tran

In [15]:
# Workflow transition putten
endpoint = f"v1/tenant/{{tenantId}}/project/{{projectId}}/acl/{{aclEntryId}}/workflow-transition-request/{request_id_search}/transitions"

steps = [
    {"transitionId": "a4ece4c3-e9a3-4a9b-b163-338c2bb04d85", "sequenceNo": 0},  # Concept → Test
    {"transitionId": "8d7442ac-56f2-4c49-922a-9d57daae9f13", "sequenceNo": 1},  # Test → Productie
]

result = client._request("PUT", endpoint, json_data={"data": steps})
print(result)


{}


In [16]:
from datetime import datetime, timezone
effectiveDate = datetime.now(timezone.utc).replace(microsecond=0).isoformat().replace("+00:00", "Z")
print(effectiveDate)

# Approves workflow transition request for specified request id
endpoint = f"v1/tenant/{{tenantId}}/project/{{projectId}}/acl/{{aclEntryId}}/workflow-transition-request/{request_id_search}/approve"
result = client._request("POST", endpoint,
                         json_data={
                             "effectiveDate": effectiveDate
                         })
print(result)

2026-01-29T11:18:20Z
{}
